# Neuromorphic Event-Based Vision Sensor

Simulating an event-based camera using a standard webcam by detecting pixel-level brightness changes over time.

## Pipeline

1. **Capture** — Webcam frame → 8-bit grayscale
2. **Difference** — Compute pixel-wise change from previous frame
3. **Threshold** — Classify each pixel as:
   - `+1` (ON) — got brighter
   - `-1` (OFF) — got darker
   - `0` — no significant change

## Threshold Tradeoff

| Threshold | Data Rate | Sensitivity |
|-----------|-----------|-------------|
| Low (5) | High | More events, more noise |
| High (20) | Low | Fewer events, cleaner output |

In [2]:
import cv2
import numpy as np

#open webcam
cap = cv2.VideoCapture(0)

In [3]:
#initializing previous frame to none since nothing has been captured yet
previous_frame = None
threshold = 20

#while loop to capture one frame at a time
while True:
    #Capture frame
    ret, frame = cap.read()
    # Convert to 8bit gray scale
    current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

   #assign first frame
    if previous_frame is None:
        previous_frame = current_frame.copy()
        continue
        
    #Create Difference frame [-255, 255]
    #use np.int64 to convert captured frame pixels to signed integers
    difference_frame = current_frame.astype(np.int64)-previous_frame.astype(np.int64) 

    #Update previous frame for subsequent iteration
    previous_frame=current_frame.copy()
    

    # windowed thresholding 
    ON=(difference_frame>threshold).astype(np.int8)*127 #if greater than threshold convert to 127
    OFF =(difference_frame<-threshold).astype(np.int8)*-128 #if less than threshold convert to -128
    NO = np.full(difference_frame.shape, 128, dtype= np.uint8) #frame of gray values assuming no event occurs
    Output =  (ON + OFF + NO).astype(np.uint8) #convert back to 8bit unsigned

    # Displaying result
    cv2.imshow("TEMPORAL CONTRAST FRAME", Output)

    # stop process when small q is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### COMMENTS

#### Reducing the threshold increases the sensitivity. A lot more motion is detected which can be very noisy in the stream. However, it enables capture of very subtle motion.

#### A higher threshold decreases the sensitivity. It takes very prominent motion to cause a change in the output. However very subtle motion may not be detected. 